# Creating a Grid of Models to use with `GridFitter`

Author:  Dominic Doud (NASA Ames Research Center)

You should already be familiar with:  
- how to output create a climate model 
- how to post process a high resolution spectral model from that climate model
- xarray file formats
- It is **very important you already understand how to analyze and diagnose the valididy of climate models**. Here we will use a streamlined function to trigger the creation of many models. However, there is always a chance that some of the climate runs do not converge. Therefore, it is important to you go back and assess each of these models. 

The main goal of this notebook will be to create a mini mini grid of models that is on a function of :

- Metallicity (mh)
- Carbon to Oxygen Ratio (cto)
- Heat redistribution (heat_redis)
- Internal Heat (tint)

Take a look at some papers which have created climate grids for recommendation on how to determine what values for each of these. Here in this notebook we will simply run two of each, just to show how the function works. 

- WASP-39 b (e.g. [Alderson et al 2023](https://ui.adsabs.harvard.edu/abs/2023Natur.614..664A/abstract))
- WASP-17 b (e.g. [Grant et al 2023](https://ui.adsabs.harvard.edu/abs/2023ApJ...956L..32G/abstract))



## Create function to run climates 

If you dont understand these inputs we encourage you to complete the basic climate tutorials first! 

In [ ]:
def run_climate(base_case_name, array_save_dir, mh, CtoO, 
                t_int, star_temp, star_metal, star_radius, 
                star_logg, semi_major, nlevel, nstr_upper,
                rfacv, opacity_ph, teq, 
                planet_mass, planet_radius,
                resume=False, nofczns=1):
    #check if done already and want to skip:
    #create file name
    print("Running climate")

    #let's create a nice naming scheme based on our parameters
    savefile=base_case_name+"_climate_tint"+str(t_int)+"_rfacv"+str(rfacv)+"_mh"+mh+"_cto"+CtoO+".nc"

    #let's make sure we dont repeat ourselves if a file already exists 
    if (resume & os.path.exists(os.path.join(array_save_dir,'climate',savefile))): 
        print('Skipping',savefile,', file already exists')
        return
    else:
        
        #This is setting up the problem for a planet 
        case_name = jdi.inputs(calculation="planet", climate = True)
        case_name.star(opacity_ph, temp=star_temp,metal=star_metal,logg=star_logg,
                        radius=star_radius, semi_major= semi_major ,
                        database='phoenix', radius_unit = jdi.u.Unit('R_sun'), semi_major_unit = jdi.u.AU)
        case_name.gravity(mass=planet_mass, mass_unit=jdi.u.Unit('M_jup'), 
                      radius=planet_radius, radius_unit=jdi.u.Unit('R_jup'))
        case_name.effective_temp(t_int)

        #setup mechanism to create initial guesses
        pt = case_name.guillot_pt(teq, nlevel=nlevel, T_int = t_int, p_bottom=2, p_top=-6) 
        temp_guess = pt['temperature'].values 
        pressure = pt['pressure'].values
    
        #time to find convection zone. let's assume it always at depth
        nstr_deep = nlevel -2 # this is always the case
        nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones
        
        case_name.inputs_climate(temp_guess=temp_guess, pressure=pressure, 
                          nstr=nstr, nofczns=nofczns, rfacv=rfacv)

        #here we are setting with_spec to False as we are going to do this ourselves later 
        out = case_name.climate(opacity_ph, save_all_profiles=True, with_spec=False) 

        #make folder if needed
        print("Saving xarray: " + savefile)
        #in add output you can add any extra meta data fields you want to have in your xarray file
        climate_xarray = jdi.output_xarray(df, case_name, add_output={'author': 'Awesome Scientist'}, 
                          savefile=os.path.join(array_save_dir,"climate",savefile))

        return savefile,climate_xarray



## Create Function to Post-Process High Res Spectra

In [ ]:
def hi_res_spec_xarr(savefile, climate_xarray, opacity_highres):
    
    #make sure we have the same naming system for this file
    spec_savefile = savefile.replace('climate','spectra')
    
    #let's make sure we dont repeat ourselves if a file already exists 
    if (resume & os.path.exists(os.path.join(array_save_dir,'spectra',spec_savefile))): 
        print('Skipping',spec_savefile,', file already exists')
        return
    else:    
        #we can directly input our climate xarray
        hi_res = jdi.input_xarray(climate_xarray,opa_mon, calculation='planet')
        #at this point you could consider post processing clouds if you wish! 
        #hi_res.clouds(... )
        df_spec = hi_res.spectrum(opacity_highres, calculation='thermal+transmission', 
                                  full_output=True)
        preserve_hires = jdi.output_xarray(df_spec, hi_res,
                                  savefile=os.path.join(array_save_dir,"spectra",spec_savefile))
        print('Done!', spec_savefile)

In [ ]:
import os
from picaso import justdoit as jdi
from picaso import justplotit as jpi
import numpy as np

## Set Inputs and Trigger Loop! 

In [ ]:
correlated_k_dir ='/data/kcoeff_2020_v3/' 
array_save_dir ='/data/test/tutorial'
mh_list = ['+000', '+100'] 
CtoO_list = ['100', '250']
rfacv_list=[0.5]
t_int_list=[200, 300]

#continue where left off? Or restart? 
resume=False

#other variables
base_case_name='wasp-39b' #tag name for filenames 
R=100 #resolution power

#wav range for high res spectra 
min_wav=3
max_wav=15

nlevel = 91 # number of atmospheric layers we want
nstr_upper = 85 #upper bound on first radiative convective boundary

#planet/star params
star_temp = 5400
star_metal = 0.01
star_radius = 0.9
star_logg = 4.45

wasp_mass=0.28
wasp_radius=1.27
teq=1120.55
semi_major=0.0486

#define what resampled opacities you want to use 
opacity_hires = jdi.opannection(wave_range=[min_wav,max_wav])


#Trigger mega loop !
for mh in mh_list:
    for CtoO in CtoO_list:
        for rfacv in rfacv_list:
            for t_int in t_int_list:
                #grab k table opacities
                print(mh, CtoO,rfacv,t_int)
                ph_db = os.path.join(
                    correlated_k_dir,f'sonora_2020_feh{mh}_co_{CtoO}_noTiOVO.data.196')
                print(ph_db)
                opacity_ph = jdi.opannection(ck_db=ph_db)
                #get climate xarray
                savefile, climate_xarray = run_climate(base_case_name, array_save_dir, mh, CtoO, t_int, star_temp, star_metal,
                            star_radius, star_logg, semi_major, nlevel, nstr_upper, rfacv, opacity_ph, teq=teq,
                            planet_mass=wasp_mass, planet_radius=wasp_radius, resume=resume)
                #get hi res xarray
                hi_res_spec_xarr(savefile,climate_xarray, opacity_hires)